In [161]:
import langchain
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import tensorflow as tf
import tensorflow_hub as hub
import sentencepiece as spm
import tiktoken

import random
import time
import openai
import os
import pinecone


In [162]:
from platform import python_version
print(python_version())

3.8.1


In [163]:
# load docs
# C:\Users\mkroh\Desktop
# alphabet_reports_for_langchain_example

loader = DirectoryLoader('c:/Users/mkroh/Desktop/alphabet_reports_for_langchain_example', 
                         glob='**/*.pdf', 
                         show_progress = True)
docs = loader.load()
# docs

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [04:21<00:00,  6.23s/it]


In [190]:
# chunk the data

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)  # OG chunk_size: 19880
docs_split = text_splitter.split_documents(docs)
# docs_split

Created a chunk of size 3329, which is longer than the specified 2000
Created a chunk of size 2874, which is longer than the specified 2000
Created a chunk of size 2521, which is longer than the specified 2000
Created a chunk of size 3362, which is longer than the specified 2000
Created a chunk of size 2018, which is longer than the specified 2000
Created a chunk of size 3949, which is longer than the specified 2000
Created a chunk of size 5521, which is longer than the specified 2000
Created a chunk of size 2088, which is longer than the specified 2000
Created a chunk of size 3481, which is longer than the specified 2000
Created a chunk of size 2872, which is longer than the specified 2000
Created a chunk of size 3265, which is longer than the specified 2000
Created a chunk of size 2168, which is longer than the specified 2000
Created a chunk of size 2335, which is longer than the specified 2000
Created a chunk of size 3473, which is longer than the specified 2000
Created a chunk of s

In [191]:
print(len(docs_split))

3287


In [192]:
# now for some pinecone, openai keys

PINECONE_API_KEY = 'get your own'
PINECONE_ENV = 'get your own'
OPENAI_API_KEY = 'get your own'

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [193]:
# docs_split_small = docs_split[:59]

# print(len(docs_split_small))

In [194]:
# print(docs_split[:1])

In [195]:
if type(docs_split) == list:
    print(True)
else:
    print(False)

True


In [196]:
# str_doc = str(docs_split[:1])
# tensor_doc = tf.convert_to_tensor(str_doc)

# print(tensor_doc)

In [197]:
# woops, can't use any of this.  awesome.  really fun.


# # convert to string, then string to tensor for embedding. 

# Check if each item in the list is a string.
is_string = [isinstance(item, str) for item in docs_split]

# Convert the items in the list to strings if they are not strings.
string_list = [str(item) if not is_string[i] else item for i, item in enumerate(docs_split)]

# # Convert the string list to tensors.
# tensor_list = [tf.convert_to_tensor(item) for item in string_list]

# # stack the tensors
# stacked_tensor = tf.stack(tensor_list)

# # print(stacked_tensor[0:3])  # check it

In [198]:
print(len(string_list))

3287


In [199]:
# print(string_list[:1])

In [200]:
# print(string_list[359:360])

In [201]:
encoding = tiktoken.get_encoding("cl100k_base")

In [202]:
encoding = tiktoken.encoding_for_model("text-embedding-ada-002")

In [203]:
encoded_strings = []
number_tokens_per_string = []

for string in string_list:
    encoded_strings.append(encoding.encode(string))  # encodes each string in string_list
    num_tokens = len(encoding.encode(string))
    number_tokens_per_string.append(num_tokens)
    

# encoding.encode(string_list[:1])

In [204]:
# validate

print(len(encoded_strings))
# print(len(encoded_strings[1]))

3287


In [205]:
# validate

print(number_tokens_per_string)

[397, 513, 502, 381, 302, 406, 614, 720, 757, 775, 360, 354, 318, 300, 482, 445, 314, 443, 649, 603, 290, 429, 561, 579, 376, 380, 496, 628, 627, 607, 517, 434, 519, 623, 718, 564, 501, 530, 363, 452, 348, 393, 351, 438, 728, 531, 424, 638, 602, 432, 371, 503, 454, 419, 442, 395, 501, 477, 418, 658, 418, 514, 393, 462, 454, 453, 446, 467, 507, 328, 435, 387, 364, 353, 394, 364, 452, 367, 400, 336, 315, 394, 314, 365, 364, 234, 340, 368, 394, 356, 369, 365, 346, 390, 319, 344, 281, 363, 372, 224, 380, 394, 315, 303, 318, 296, 384, 408, 413, 328, 231, 380, 349, 411, 359, 328, 403, 426, 362, 349, 350, 298, 365, 422, 441, 394, 538, 314, 385, 516, 502, 379, 299, 406, 609, 795, 807, 808, 776, 425, 364, 345, 366, 499, 647, 314, 442, 690, 622, 393, 571, 658, 453, 341, 407, 577, 634, 669, 648, 673, 406, 409, 610, 570, 735, 710, 598, 389, 349, 443, 364, 376, 351, 441, 801, 690, 481, 433, 703, 634, 465, 413, 353, 583, 474, 472, 431, 279, 434, 544, 572, 364, 768, 603, 491, 452, 450, 441, 427, 436,

In [206]:
print(max(number_tokens_per_string))

6943


In [207]:
print(number_tokens_per_string.index(max(number_tokens_per_string)))

1867


In [208]:
# or, a function


# def find_max_index(number_tokens_per_string):
#   max_value = max(number_tokens_per_string)
#   max_value_index = number_tokens_per_string.index(max_value)

#   return max_value_index

# find_max_index

# if __name__ == "__main__":
#   max_value_index = find_max_index(number_tokens_per_string)
#   print("The index of the maximum value is", max_value_index)

In [209]:
# verify

print(len(encoded_strings[number_tokens_per_string.index(max(number_tokens_per_string))]))

6943


In [210]:
# print(encoded_strings[1])

In [211]:
# print(encoded_strings[100])

In [212]:
# lol, can't tell if they match!

if encoded_strings[1] == encoded_strings[100]:
    print("True")
else:
    print("False and phew")


False and phew


In [213]:
# from openai cookbook.  determine cost per number of tokens with this encoder, tokenizer

cost_per_thousand_tokens = 0.0001


max_tokens = max(number_tokens_per_string)
max_strings_sent = len(encoded_strings)

max_num_tokens_all_strings = max_tokens*max_strings_sent

cost = max_num_tokens_all_strings*cost_per_thousand_tokens/1000

print("Maximum possible tokens", max_num_tokens_all_strings)
print("Worst case for costing (in dollars):", cost)

Maximum possible tokens 22821641
Worst case for costing (in dollars): 2.2821641


Commentary: just to make a point that processing matters.  On the OpenAI side of things, by changing the chunks to a size of 19880 or so will result in an overall lower cost: 0.36 versus 4.41 (chunk_size: 1000) or 7.53 (chunk_size: 500).

I might hit another limit anyway because the max token length I need to send exceeds 2000 tokens/request.

In [214]:
# we use the openAI embedding model
embeddings = OpenAIEmbeddings()
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

doc_db = Pinecone.from_documents(docs_split, embeddings, index_name='langchain-example', batch_size = 50)

In [215]:
# # had to switch from openai embedding to google universal sentence encoder due to data rate limitations on a free account.

# # # load Universal Sentence Encoder
# # embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# # # converted to tensor list above, now embed
# # embeddings = embed(stacked_tensor)

# # load sentencepiece model
# sp = spm.SentencePieceProcessor()
# sp.Load('sentencepiece.model')

# # convert list of docs to strings
# string_list = [doc.strip() for doc in docs_split]

# # get embeddings for docs
# embeddings = sp.PieceToId(string_list)

# # Initialize Pinecone
# pinecone.init(api_key = PINECONE_API_KEY, environment = PINECONE_ENV)

# # create Pinecone db from docs and embeddings
# doc_db = Pinecone.from_documents(docs_split, embeddings, index_name = 'langchain-example', batch_size = 6) 

In [217]:
query = "What were the most important events for Google in 2021?"
search_docs = doc_db.similarity_search(query)
search_docs

[Document(page_content='When the world shifted to learning and working from home, Google data centers kept us running, supported users, and, crucially, supported our partners. Whether our partners are developers, advertisers, content creators, or merchants, our performance is only made possible by their success.\n\nFor many of our customers, digital transformation in the cloud became an urgent business priority in 2020. Last year, Google hosted over a trillion minutes of video meetings and over 2.9 billion users chose productivity apps like Gmail, Calendar, Drive, Docs, Sheets, Slides, and Meet every single day.\n\nDevelopers were behind the apps that kept people\n\nentertained and connected while at home. To ensure developers could access the latest information about our products, and could connect with us and each other, we pivoted all our major conferences online, including 11 weeks of developer content to support the launch of Android 11 Beta. This gave millions of developers easy 

In [227]:
# llm to utilize vector database data

# llm = OpenAI()

llm = ChatOpenAI()  # alternative.  ChatGPT is this one.

In [228]:
# query the data with retrievalQA

qa = RetrievalQA.from_chain_type(llm = llm, chain_type = 'stuff', retriever=doc_db.as_retriever())

query = "What were the earnings in 2022?"
result = qa.run(query)
result

In [229]:
result

'The total revenues for the year ended December 31, 2022, were $282,836 million. However, the context does not provide information on the specific earnings for the year.'

In [232]:
query = "What did Google focus on in 2022?"
result = qa.run(query)
result

'The given text does not provide information about what Google focused on in 2022 as it is a hypothetical year and has not happened yet. However, the text does mention that Google strives to empower people, businesses, and communities with technology that can help them navigate our changing world. In 2021, Google continued to invest deeply in AI and other technologies to ensure the most helpful search experience possible, and also worked to advance knowledge and computing, drawing on AI to build products that are more accessible and helpful for everyone, everywhere. Additionally, Google added new languages to Google Translate, introduced auto-translated captions on YouTube, and added live busyness trends in Google Maps to help users instantly spot when a neighborhood or part of town is near or at its busiest.'